In [103]:
import sys
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import plotly.express as px
%matplotlib inline

In [104]:
parent_dir = os.path.abspath(r'D:')
raw_data_dir = os.path.join(parent_dir, 'raw')
#processed_data_dir = os.path.join(parent_dir, 'processed_data')
#concatenated_yaws_dir = os.path.join(parent_dir, 'Concatenated_Yaws')


In [105]:
%%time
os.chdir(r'C:\Users\musab\raw\Yaw00')
all_files=glob.glob('*int')
print(all_files)

['0001.int', '0002.int', '0003.int', '0004.int', '0005.int']
Wall time: 1.99 ms



dir_path =parent_dir

items = os.listdir(dir_path)

# Filter out only the directories
folders = [item for item in items if os.path.isdir(os.path.join(dir_path, item))]

for folder_name in folders:
    folder_path = os.path.join(dir_path, folder_name)
    os.chdir(folder_path)
    all_files = glob.glob('*int')
    print(f"Processing {len(all_files)} files in folder: {folder_name}")
    print(all_files)


In [106]:

sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)


  #print(" 0 1 2  3  4 5  6  7 8  9 10 11 12  13  14 15  16 17   18   19  20")
    
def read_int(FileName,x):

    #print(FileName)

    xlen = len(FileName)
    ending = FileName[xlen-4:xlen]

    if ending=='.int':
        FileName = FileName[0:xlen-4]


    with open(FileName+ending,"rb") as f:
        f.read()
        filesize = f.tell()
       # print(filesize)
       

    f.close()




    with open(FileName+ending,"rb") as f:

    # the first 4 junk buffer
        
        leer = np.frombuffer(f.read(4),dtype=np.single)
        #print(leer)
    # the  24 buffer give information about file
        datei = np.frombuffer(f.read(24),dtype=np.uint32)
        #print(datei)

        f.seek(76)
    # read the number of sensor in file 
        lvek = np.frombuffer(f.read(4),dtype=np.uint32)
        #print(lvek)
     
    # then 4 times the number of sensors are skipped to read a control parameter:
  
        f.seek(84+lvek[0]*4)

        filetype = np.frombuffer(f.read(4),dtype=np.uint)
        #print(filetype)

        if filetype != 12:

    # start time is read (is not equal to 0 as we have been cut off from the simulation for the first 50 s
            t0 = np.frombuffer(f.read(4),dtype=np.single)
            #print(t0)

    # comes the parameter for the step size of the simulation

            dt = np.frombuffer(f.read(4),dtype=np.single)
            #print(dt)

            pos_fak = f.tell()
            #print(pos_fak)

            fak = []
 
            for n in x:
                f.seek(pos_fak + 4*(n-1))
                fak.append(np.frombuffer(f.read(4),dtype=np.single))
                
            f.seek(pos_fak+lvek[0]*4)
            position = f.tell()
    # the length of the sensors is calculated:
            RecordCount = round((filesize-position)/lvek[0]/2)
            #print(RecordCount)

            raw_ts_int = np.empty([len(x),RecordCount])
           # print(raw_ts_int)
        
            #print("Type:",type(raw_ts_int))
            #print(raw_ts_int.shape)

            for n in range(RecordCount):

                for m, sensor_ in enumerate(x):

                    f.seek(position+2*sensor_+(lvek[0]*2)*n-2)
                    raw_ts_int[m,n] = np.frombuffer(f.read(2),dtype=np.int16)*fak[m]

            global   df
            df = pd.DataFrame(raw_ts_int)
            df=df.transpose()
            #print(df)
            #print(df.transpose())
            #merged_df=df.merge(df)
                    
    f.close()


In [67]:
for file in all_files:
    if file.endswith('.int'):
        read_int(file, sensors)


             0         1             2             3             4   \
0      3.926823  0.188507  1.142097e-13  7.815725e-14  1.983951e-13   
1      3.946518  0.615087  1.138299e-13  7.775590e-14  1.981307e-13   
2      3.935961 -0.054574  1.133397e-13  7.729511e-14  1.977528e-13   
3      3.987010 -0.282134  1.126994e-13  7.662621e-14  1.971861e-13   
4      3.992840  0.179995  1.120988e-13  7.598703e-14  1.966194e-13   
...         ...       ...           ...           ...           ...   
19996  4.087217 -1.499290  5.054936e-14 -4.568438e-13  1.207940e-12   
19997  4.111796 -1.887569  5.026674e-14 -4.571708e-13  1.207713e-12   
19998  4.088635 -1.570387  4.992671e-14 -4.575127e-13  1.207486e-12   
19999  4.135745 -1.266974  4.962642e-14 -4.578249e-13  1.207222e-12   
20000  4.140787 -0.990097  4.944537e-14 -4.580033e-13  1.207033e-12   

               5           6         7         8          9   ...        11  \
0      341.045624  620.029236  7.797325  1.483438  96.298790  ... -0

             0         1             2             3             4   \
0      4.187558  0.922236  1.224818e-13  8.282042e-14  2.053291e-13   
1      4.205194  0.241555  1.225270e-13  8.283540e-14  2.054451e-13   
2      4.205036  0.159174  1.225586e-13  8.280543e-14  2.054838e-13   
3      4.206295  0.432321  1.226671e-13  8.286538e-14  2.055997e-13   
4      4.188975  0.745322  1.228208e-13  8.298527e-14  2.057930e-13   
...         ...       ...           ...           ...           ...   
19996  4.149769  2.223573  6.422492e-14 -4.459203e-13  1.236034e-12   
19997  4.130086  2.427461  6.429275e-14 -4.458904e-13  1.236150e-12   
19998  4.140321  2.786392  6.432439e-14 -4.458604e-13  1.236227e-12   
19999  4.132291  2.862455  6.434701e-14 -4.458754e-13  1.236305e-12   
20000  4.160948  3.019199  6.438317e-14 -4.458754e-13  1.236343e-12   

               5           6         7         8          9   ...        11  \
0      317.435822  615.595642  7.741111  1.317304  84.912155  ... -0

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,4.670816,-3.088937,1.114139e-13,7.440504e-14,1.954890e-13,40.904892,625.065796,7.860678,1.672114,109.460190,...,-0.082080,0.075402,0.018292,0.060730,0.511489,0.574599,0.456535,0.190535,0.038427,-0.510280
1,4.689983,-2.680327,1.111852e-13,7.412834e-14,1.952954e-13,42.311138,625.030334,7.860678,1.671134,109.402969,...,-0.081899,0.076209,0.018487,0.060370,0.495841,0.584150,0.453229,0.200023,0.028219,-0.509674
2,4.690674,-3.008855,1.110137e-13,7.392445e-14,1.952180e-13,43.728638,624.994812,7.861125,1.670154,109.288528,...,-0.087745,0.080268,0.018607,0.060080,0.479898,0.593774,0.451906,0.208728,0.018319,-0.509105
3,4.646122,-3.481386,1.108642e-13,7.374968e-14,1.951405e-13,45.146133,624.959351,7.861125,1.668684,109.231308,...,-0.082123,0.075764,0.018650,0.059869,0.464029,0.603325,0.452641,0.216602,0.008790,-0.508535
4,4.634553,-3.093761,1.107455e-13,7.358948e-14,1.950631e-13,46.563629,624.959351,7.861570,1.667704,109.174088,...,-0.085319,0.072844,0.018617,0.059720,0.448821,0.612729,0.455139,0.223599,-0.000370,-0.507787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,4.043988,2.248561,5.466916e-14,-4.339785e-13,1.238381e-12,262.551819,631.237366,7.940539,1.887744,124.794914,...,0.009576,0.040370,-0.008361,0.073156,0.358895,0.450363,0.726679,-0.530785,0.042467,0.189027
19997,4.040707,2.238671,5.454603e-14,-4.341532e-13,1.238304e-12,263.980560,631.237366,7.940093,1.888725,124.852135,...,0.007193,0.024550,-0.008470,0.073490,0.359336,0.451098,0.715512,-0.531567,0.052522,0.179697
19998,4.040017,2.550073,5.454603e-14,-4.341678e-13,1.238381e-12,265.409302,631.237366,7.940539,1.889215,124.852135,...,0.009362,0.019740,-0.008571,0.073841,0.360511,0.450510,0.704271,-0.532184,0.062884,0.169939
19999,4.019986,1.876132,5.458121e-14,-4.342115e-13,1.238420e-12,266.838043,631.272827,7.940985,1.889215,124.909348,...,0.009394,0.016654,-0.008664,0.074219,0.362568,0.448894,0.693398,-0.532540,0.073586,0.159754


In [60]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Set the batch size (number of records to process at a time)
batch_size = 1000

# Set the sensor numbers you want to process
sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

# Set the random seed for reproducibility
random_seed = 42



# Set the directory containing .int files
os.chdir(r'C:\Users\musab\raw\Yaw00')

# Get the list of all .int files
all_files = glob.glob('*.int')

# List to store sampled dataframes
sampled_dfs = []

# Process each file
for file in all_files:
    if file.endswith('.int'):
        read_int(file, sensors)
        sampled_dfs.append(df.sample(frac=0.1, random_state=random_seed))  # Randomly sample 10% of the data

# Concatenate the sampled dataframes into a single dataframe
df_sampled = pd.concat(sampled_dfs)

# Transpose the dataframe
df_sampled_transposed = df_sampled.T

# Split the data into training and testing sets
train_df, test_df = train_test_split(df_sampled_transposed, test_size=0.2, random_state=random_seed)

# Perform iterative modeling
models = []
for i in range(10):
    # Train the model
    model = LinearRegression()
    X_train = train_df.drop(columns=[target_column])
    y_train = train_df[target_column]
    model.fit(X_train, y_train)

    # Evaluate the model on the test set
    X_test = test_df.drop(columns=[target_column])
    y_test = test_df[target_column]
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    print(f"Iteration {i+1}: MSE = {mse}")

    models.append(model)

# Perform final evaluation on the full test set
X_test_full = df_sampled_transposed.drop(columns=[target_column])
y_test_full = df_sampled_transposed[target_column]

final_mses = []
for model in models:
    y_pred_full = model.predict(X_test_full)
    mse = mean_squared_error(y_test_full, y_pred_full)
    final_mses.append(mse)

average_mse = np.mean(final_mses)
print(f"Average MSE on full test set: {average_mse}")


NameError: name 'target_column' is not defined

In [6]:
%%time
read_int('0001.int',sensor)
raw_1stfile=df.transpose()
raw_1stfile

Wall time: 1.74 s


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3.926823,0.188507,1.142097e-13,7.815725e-14,1.983951e-13,341.045624,620.029236,7.797325,1.483438,96.298790,-0.076369,0.024555,0.018018,0.058451,0.422960,0.776565,0.615227,-0.245652,0.344958,-0.374269
1,3.946518,0.615087,1.138299e-13,7.775590e-14,1.981307e-13,342.451813,620.029236,7.797771,1.483928,96.356010,-0.079791,0.026061,0.018295,0.057719,0.421417,0.777814,0.605823,-0.233487,0.341019,-0.383040
2,3.935961,-0.054574,1.133397e-13,7.729511e-14,1.977528e-13,343.858032,620.064697,7.799110,1.484418,96.356010,-0.082149,0.047319,0.018497,0.057026,0.419800,0.778622,0.596125,-0.221180,0.336927,-0.391281
3,3.987010,-0.282134,1.126994e-13,7.662621e-14,1.971861e-13,345.252991,620.064697,7.799110,1.485398,96.470444,-0.081660,0.040101,0.018627,0.056362,0.418478,0.778769,0.586648,-0.208801,0.332774,-0.398956
4,3.992840,0.179995,1.120988e-13,7.598703e-14,1.966194e-13,346.659210,620.100159,7.800002,1.485888,96.470444,-0.080693,0.049564,0.018685,0.055745,0.417449,0.778181,0.577758,-0.196493,0.328651,-0.406171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,4.087217,-1.499290,5.054936e-14,-4.568438e-13,1.207940e-12,283.705933,607.402344,7.640281,0.819885,52.126083,0.008506,-0.005030,-0.007783,0.048411,0.718524,0.664672,0.904841,-0.479568,0.205857,0.046439
19997,4.111796,-1.887569,5.026674e-14,-4.571708e-13,1.207713e-12,285.078400,607.402344,7.640728,0.820865,52.240520,0.008528,0.009805,-0.007764,0.047778,0.731161,0.653211,0.900874,-0.476835,0.213003,0.035970
19998,4.088635,-1.570387,4.992671e-14,-4.575127e-13,1.207486e-12,286.450867,607.437805,7.640728,0.823315,52.354958,0.008702,0.014608,-0.007737,0.047153,0.743871,0.642117,0.895951,-0.474127,0.220058,0.025748
19999,4.135745,-1.266974,4.962642e-14,-4.578249e-13,1.207222e-12,287.823334,607.437805,7.640728,0.825766,52.526615,0.009169,0.006423,-0.007702,0.046535,0.756140,0.631390,0.889853,-0.471561,0.226929,0.015774


In [17]:
import os
import glob
import pandas as pd
import random

os.chdir(r'C:\Users\musab\raw\Yaw00')  # Change to your desired directory
all_files = glob.glob('*int')

sensor = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)
combined_data = []

sample_size = 100  # Adjust the sample size as needed

for file in all_files:
    read_int(file, sensor)
    if not df.empty:
        if df.shape[0] > sample_size:
            sample = df.sample(n=sample_size, random_state=42)  # Select a random sample
        else:
            sample = df  # Use the entire data if it's smaller than the desired sample size
        combined_data.append(sample)

combined_data = pd.concat(combined_data, axis=1)



In [18]:
combined_data

,0,1,2,3,4,5,6,7,8,9,...,19991,19992,19993,19994,19995,19996,19997,19998,19999,20000
0,3.926823e+00,3.946518e+00,3.935961e+00,3.987010e+00,3.992840e+00,4.004184e+00,3.973145e+00,3.964795e+00,3.926666e+00,3.906498e+00,...,4.042779e+00,3.982341e+00,3.971462e+00,3.980787e+00,4.040707e+00,4.043988e+00,4.040707e+00,4.040017e+00,4.019986e+00,4.067472e+00
1,1.885065e-01,6.150871e-01,-5.457427e-02,-2.821340e-01,1.799950e-01,-1.877555e-02,-3.634947e-01,-8.511584e-03,6.408722e-02,-7.780589e-01,...,3.015609e+00,2.635221e+00,1.849358e+00,2.607723e+00,2.877637e+00,2.248561e+00,2.238671e+00,2.550073e+00,1.876132e+00,1.872755e+00
2,1.142097e-13,1.138299e-13,1.133397e-13,1.126994e-13,1.120988e-13,1.116749e-13,1.115424e-13,1.115910e-13,1.117102e-13,1.118604e-13,...,5.575093e-14,5.556184e-14,5.531558e-14,5.506053e-14,5.487584e-14,5.466916e-14,5.454603e-14,5.454603e-14,5.458121e-14,5.465596e-14
3,7.815725e-14,7.775590e-14,7.729511e-14,7.662621e-14,7.598703e-14,7.543705e-14,7.524381e-14,7.525868e-14,7.527354e-14,7.546678e-14,...,-4.328279e-13,-4.330318e-13,-4.332211e-13,-4.335561e-13,-4.337600e-13,-4.339785e-13,-4.341532e-13,-4.341678e-13,-4.342115e-13,-4.341969e-13
4,1.983951e-13,1.981307e-13,1.977528e-13,1.971861e-13,1.966194e-13,1.962038e-13,1.960904e-13,1.961660e-13,1.963171e-13,1.966194e-13,...,1.239117e-12,1.239001e-12,1.238807e-12,1.238614e-12,1.238497e-12,1.238381e-12,1.238304e-12,1.238381e-12,1.238420e-12,1.238536e-12
5,3.410456e+02,3.424518e+02,3.438580e+02,3.452530e+02,3.466592e+02,3.480654e+02,3.494716e+02,3.508778e+02,3.522728e+02,3.536790e+02,...,2.554081e+02,2.568368e+02,2.582656e+02,2.596943e+02,2.611231e+02,2.625518e+02,2.639806e+02,2.654093e+02,2.668380e+02,2.677943e+02
6,6.200292e+02,6.200292e+02,6.200647e+02,6.200647e+02,6.201002e+02,6.201002e+02,6.201002e+02,6.201002e+02,6.200647e+02,6.200292e+02,...,6.311309e+02,6.311309e+02,6.311309e+02,6.311664e+02,6.312018e+02,6.312374e+02,6.312374e+02,6.312374e+02,6.312728e+02,6.312728e+02
7,7.797325e+00,7.797771e+00,7.799110e+00,7.799110e+00,7.800002e+00,7.800002e+00,7.800448e+00,7.800448e+00,7.800448e+00,7.800448e+00,...,7.940093e+00,7.940093e+00,7.939646e+00,7.940539e+00,7.940539e+00,7.940539e+00,7.940093e+00,7.940539e+00,7.940985e+00,7.940539e+00
8,1.483438e+00,1.483928e+00,1.484418e+00,1.485398e+00,1.485888e+00,1.486378e+00,1.486378e+00,1.485888e+00,1.485398e+00,1.483928e+00,...,1.884314e+00,1.884804e+00,1.885294e+00,1.885784e+00,1.886764e+00,1.887744e+00,1.888725e+00,1.889215e+00,1.889215e+00,1.889705e+00
9,9.629879e+01,9.635601e+01,9.635601e+01,9.647044e+01,9.647044e+01,9.652766e+01,9.652766e+01,9.647044e+01,9.641323e+01,9.635601e+01,...,1.245660e+02,1.245660e+02,1.246233e+02,1.246233e+02,1.247377e+02,1.247949e+02,1.248521e+02,1.248521e+02,1.249093e+02,1.249093e+02


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Set the random seed for reproducibility
random_seed = 42

# Set the batch size (number of records to process at a time)
batch_size = 1000

# Set the sensor numbers you want to process
sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

# Read the .int files and create the dataframe
os.chdir(r'C:\Users\musab\raw\Yaw00')
all_files = glob.glob('*int')
dataframes = []

def read_int(FileName, x):
    # ... (Your existing code for reading .int files and creating dataframe)

    dataframes.append(df)

# Process the files in smaller batches
for file in all_files:
    if file.endswith('.int'):
        read_int(file, sensors)

# Concatenate all dataframes into a single dataframe
df = pd.concat(dataframes)

# Transpose the dataframe
df = df.transpose()

# Set the target columns
target_columns = [0, 10, 13, 14, 15]

# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=random_seed)

# Perform iterative modeling
models = []
rmse_scores = []

for i in range(len(target_columns)):
    # Train the model
    model = LinearRegression()
    X_train = train_df.drop(columns=target_columns)
    y_train = train_df[target_columns[i]]
    model.fit(X_train, y_train)

    # Evaluate on the test set
    X_test = test_df.drop(columns=target_columns)
    y_test = test_df[target_columns[i]]
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    models.append(model)
    rmse_scores.append(rmse)

# Print RMSE scores for each target column
for i, column in enumerate(target_columns):
    print(f"Target Column {column}: RMSE = {rmse_scores[i]}")


In [75]:
def read_int(FileName, x):

    xlen = len(FileName)
    ending = FileName[xlen-4:xlen]

    if ending == '.int':
        FileName = FileName[0:xlen-4]

    with open(FileName + ending, "rb") as f:
        f.read()
        filesize = f.tell()

    with open(FileName + ending, "rb") as f:
        leer = np.frombuffer(f.read(4), dtype=np.single)
        datei = np.frombuffer(f.read(24), dtype=np.uint32)

        f.seek(76)
        lvek = np.frombuffer(f.read(4), dtype=np.uint32)

        f.seek(84 + lvek[0] * 4)
        filetype = np.frombuffer(f.read(4), dtype=np.uint)

        if filetype != 12:
            t0 = np.frombuffer(f.read(4), dtype=np.single)
            dt = np.frombuffer(f.read(4), dtype=np.single)

            pos_fak = f.tell()
            fak = []

            for n in x:
                f.seek(pos_fak + 4 * (n - 1))
                fak.append(np.frombuffer(f.read(4), dtype=np.single))

            f.seek(pos_fak + lvek[0] * 4)
            position = f.tell()
            RecordCount = round((filesize - position) / lvek[0] / 2)

            raw_ts_int = np.empty([RecordCount, len(x)])

            for n in range(RecordCount):
                for m, sensor_ in enumerate(x):
                    f.seek(position + 2 * sensor_ + (lvek[0] * 2) * n - 2)
                    raw_ts_int[n, m] = np.frombuffer(f.read(2), dtype=np.int16) * fak[m]

            df = pd.DataFrame(raw_ts_int, columns=x)
            

    return df.transpose()


In [84]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the sensor columns
sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)



# Set the directory containing the .int files
int_directory = r'C:\Users\musab\raw\Yaw00'
os.chdir(int_directory)

# Get a list of all .int files in the directory
all_files = glob.glob('*.int')

dfs = []
for file in all_files:
    if file.endswith('.int'):
        df = read_int(file, sensors)
        df = df.transpose()
        dfs.append(df)

# Concatenate the data from all .int files into a single DataFrame
data = pd.concat(dfs)

# Transpose the DataFrame
data = data.transpose()

# Define the target sensors
target_sensors = [1, 45, 56, 57]

# Split the data into input (X) and target (y) variables
X = data.drop(columns=target_sensors)
y = data[target_sensors[1]]  # Assuming the first target sensor is the target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)


Mean Squared Error (MSE): 0.2998681711723131
Mean Absolute Error (MAE): 0.27168638509247095
Root Mean Squared Error (RMSE): 0.5476022015773065
R-squared (R2): 0.858758424420693


In [87]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set the random seed for reproducibility
random_seed = 42

# Set the batch size (number of records to process at a time)
batch_size = 1000

# Set the sensor numbers you want to process
sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

# Read the .int files and create the dataframe
os.chdir(r'C:\Users\musab\raw\Yaw00')
all_files = glob.glob('*int')
dataframes = []

def read_int(FileName,x):

    #print(FileName)

    xlen = len(FileName)
    ending = FileName[xlen-4:xlen]

    if ending=='.int':
        FileName = FileName[0:xlen-4]


    with open(FileName+ending,"rb") as f:
        f.read()
        filesize = f.tell()
       # print(filesize)
       

    f.close()




    with open(FileName+ending,"rb") as f:

    # the first 4 junk buffer
        
        leer = np.frombuffer(f.read(4),dtype=np.single)
        #print(leer)
    # the  24 buffer give information about file
        datei = np.frombuffer(f.read(24),dtype=np.uint32)
        #print(datei)

        f.seek(76)
    # read the number of sensor in file 
        lvek = np.frombuffer(f.read(4),dtype=np.uint32)
        #print(lvek)
     
    # then 4 times the number of sensors are skipped to read a control parameter:
  
        f.seek(84+lvek[0]*4)

        filetype = np.frombuffer(f.read(4),dtype=np.uint)
        #print(filetype)

        if filetype != 12:

    # start time is read (is not equal to 0 as we have been cut off from the simulation for the first 50 s
            t0 = np.frombuffer(f.read(4),dtype=np.single)
            #print(t0)

    # comes the parameter for the step size of the simulation

            dt = np.frombuffer(f.read(4),dtype=np.single)
            #print(dt)

            pos_fak = f.tell()
            #print(pos_fak)

            fak = []
 
            for n in x:
                f.seek(pos_fak + 4*(n-1))
                fak.append(np.frombuffer(f.read(4),dtype=np.single))
                
            f.seek(pos_fak+lvek[0]*4)
            position = f.tell()
    # the length of the sensors is calculated:
            RecordCount = round((filesize-position)/lvek[0]/2)
            #print(RecordCount)

            raw_ts_int = np.empty([len(x),RecordCount])
           # print(raw_ts_int)
        
            #print("Type:",type(raw_ts_int))
            #print(raw_ts_int.shape)

            for n in range(RecordCount):

                for m, sensor_ in enumerate(x):

                    f.seek(position+2*sensor_+(lvek[0]*2)*n-2)
                    raw_ts_int[m,n] = np.frombuffer(f.read(2),dtype=np.int16)*fak[m]

            global   df
            df = pd.DataFrame(raw_ts_int)
            df=df.transpose()
            #print(df)
            #print(df.transpose())
            #merged_df=df.merge(df)
            dataframes.append(df)        
    f.close()


    

# Process the files in smaller batches
for file in all_files:
    if file.endswith('.int'):
        read_int(file, sensors)

# Concatenate all dataframes into a single dataframe
df = pd.concat(dataframes)

# Transpose the dataframe
df = df.transpose()

# Set the target columns
target_columns = [1, 45, 56, 57]

# Perform iterative modeling
models = []
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []

for i in range(len(target_columns)):
    # Split the data into train and test sets
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=random_seed)

    # Train the model
    model = LinearRegression()
    X_train = train_df.drop(columns=target_columns)
    y_train = train_df[target_columns[i]]
    model.fit(X_train, y_train)

    # Evaluate on the test set
    X_test = test_df.drop(columns=target_columns)
    y_test = test_df[target_columns[i]]
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    models.append(model)
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

# Print evaluation scores for each target column
for i, column in enumerate(target_columns):
    print(f"Target Column {column}:")
    print("Mean Squared Error (MSE):", mse_scores[i])
    print("Mean Absolute Error (MAE):", mae_scores[i])
    print("Root Mean Squared Error (RMSE):", rmse_scores[i])
    print("R-squared (R2):", r2_scores[i])
    print()


Target Column 1:
Mean Squared Error (MSE): 0.5815690472954741
Mean Absolute Error (MAE): 0.39937634149327167
Root Mean Squared Error (RMSE): 0.7626067448531216
R-squared (R2): 0.859199897684845

Target Column 45:
Mean Squared Error (MSE): 0.36877372990179724
Mean Absolute Error (MAE): 0.3248436088170783
Root Mean Squared Error (RMSE): 0.6072674286521526
R-squared (R2): 0.8700380072733036

Target Column 56:
Mean Squared Error (MSE): 0.3728160531389756
Mean Absolute Error (MAE): 0.2822523449581939
Root Mean Squared Error (RMSE): 0.6105866467087007
R-squared (R2): 0.8426185862806005

Target Column 57:
Mean Squared Error (MSE): 0.6040038617075463
Mean Absolute Error (MAE): 0.32559578484756724
Root Mean Squared Error (RMSE): 0.7771768535587934
R-squared (R2): 0.7626906960460186



In [90]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set the random seed for reproducibility
random_seed = 42

# Set the batch size (number of records to process at a time)
batch_size = 1000

# Set the sensor numbers you want to process
sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

# Read the .int files and create the dataframe
os.chdir(r'C:\Users\musab\raw\Yaw00')
all_files = glob.glob('*int')
dataframes = []



# Process the files in smaller batches
for file in all_files:
    if file.endswith('.int'):
        read_int(file, sensors)  # Pass the sensors variable to the read_int function

# Concatenate all dataframes into a single dataframe
df = pd.concat(dataframes)

# Transpose the dataframe
df = df.transpose()

# Set the target columns
target_columns = [1, 45, 56, 57]

# Perform iterative modeling
models = []
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []

for i in range(len(target_columns)):
    # Split the data into train and test sets
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=random_seed)

    # Train the model
    model = RandomForestRegressor(random_state=random_seed)
    X_train = train_df.drop(columns=target_columns)
    y_train = train_df[target_columns[i]]
    model.fit(X_train, y_train)

    # Evaluate on the test set
    X_test = test_df.drop(columns=target_columns)
    y_test = test_df[target_columns[i]]
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    models.append(model)
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

# Print evaluation scores for each target column
for i, column in enumerate(target_columns):
    print(f"Target Column {column}:")
    print("Mean Squared Error (MSE):", mse_scores[i])
    print("Mean Absolute Error (MAE):", mae_scores[i])
    print("Root Mean Squared Error (RMSE):", rmse_scores[i])
    print("R-squared (R2):", r2_scores[i])
    print()


Target Column 1:
Mean Squared Error (MSE): 2.035981087913047
Mean Absolute Error (MAE): 1.0060449434304648
Root Mean Squared Error (RMSE): 1.4268780914685903
R-squared (R2): 0.48746611642941523

Target Column 45:
Mean Squared Error (MSE): 1.6144375265268949
Mean Absolute Error (MAE): 0.9320013697594387
Root Mean Squared Error (RMSE): 1.2706051812136194
R-squared (R2): 0.505453207105346

Target Column 56:
Mean Squared Error (MSE): 1.549448119404321
Mean Absolute Error (MAE): 0.9473317022439979
Root Mean Squared Error (RMSE): 1.2447682994856195
R-squared (R2): 0.5084217935664702

Target Column 57:
Mean Squared Error (MSE): 1.723070503947832
Mean Absolute Error (MAE): 0.9889354415000369
Root Mean Squared Error (RMSE): 1.312657801541526
R-squared (R2): 0.47076545184021656



In [97]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#random_seeds = [42, 123, 456]  
#batch_sizes = [1000, 2000, 5000]  
random_seeds = [42]
batch_sizes = [1000]



sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

os.chdir(r'C:\Users\musab\raw\Yaw00')
all_files = glob.glob('*int')
dataframes = []



for seed in random_seeds:
    for batch_size in batch_sizes:
        print(f"Iterative Modeling: Random Seed = {seed}, Batch Size = {batch_size}")

        for file in all_files:
            if file.endswith('.int'):
                read_int(file, sensors) 
        df = pd.concat(dataframes)

        df = df.transpose()

        target_columns = [1, 45, 56, 57]

        models = []
        mse_scores = []
        mae_scores = []
        rmse_scores = []
        r2_scores = []

        for i in range(len(target_columns)):
            train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)

            model = RandomForestRegressor(random_state=seed)
            X_train = train_df.drop(columns=target_columns)
            y_train = train_df[target_columns[i]]
            model.fit(X_train, y_train)

            X_test = test_df.drop(columns=target_columns)
            y_test = test_df[target_columns[i]]
            y_pred = model.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            models.append(model)
            mse_scores.append(mse)
            mae_scores.append(mae)
            rmse_scores.append(rmse)
            r2_scores.append(r2)

        for i, column in enumerate(target_columns):
            print(f"Target Column {column}:")
            print("Mean Squared Error (MSE):", mse_scores[i])
            print("Mean Absolute Error (MAE):", mae_scores[i])
            print("Root Mean Squared Error (RMSE):", rmse_scores[i])
            print("R-squared (R2):", r2_scores[i])
            print()


Iterative Modeling: Random Seed = 42, Batch Size = 1000
Target Column 1:
Mean Squared Error (MSE): 2.035981087913047
Mean Absolute Error (MAE): 1.0060449434304648
Root Mean Squared Error (RMSE): 1.4268780914685903
R-squared (R2): 0.48746611642941523

Target Column 45:
Mean Squared Error (MSE): 1.6144375265268949
Mean Absolute Error (MAE): 0.9320013697594387
Root Mean Squared Error (RMSE): 1.2706051812136194
R-squared (R2): 0.505453207105346

Target Column 56:
Mean Squared Error (MSE): 1.549448119404321
Mean Absolute Error (MAE): 0.9473317022439979
Root Mean Squared Error (RMSE): 1.2447682994856195
R-squared (R2): 0.5084217935664702

Target Column 57:
Mean Squared Error (MSE): 1.723070503947832
Mean Absolute Error (MAE): 0.9889354415000369
Root Mean Squared Error (RMSE): 1.312657801541526
R-squared (R2): 0.47076545184021656



In [99]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

random_seeds = [42]
batch_sizes = [1000]

sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12,45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

os.chdir(r'C:\Users\musab\raw\Yaw00')
all_files = glob.glob('*int')
dataframes = []

evaluation_scores = []

for seed in random_seeds:
    for batch_size in batch_sizes:
        print(f"Iterative Modeling: Random Seed = {seed}, Batch Size = {batch_size}")

        for file in all_files:
            if file.endswith('.int'):
                read_int(file, sensors) 
        df = pd.concat(dataframes)

        df = df.transpose()

        target_columns = [1, 45, 56, 57]

        models = []
        mse_scores = []
        mae_scores = []
        rmse_scores = []
        r2_scores = []

        for i in range(len(target_columns)):
            train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)

            model = RandomForestRegressor(random_state=seed)
            X_train = train_df.drop(columns=target_columns)
            y_train = train_df[target_columns[i]]
            model.fit(X_train, y_train)

            X_test = test_df.drop(columns=target_columns)
            y_test = test_df[target_columns[i]]
            y_pred = model.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            models.append(model)
            mse_scores.append(mse)
            mae_scores.append(mae)
            rmse_scores.append(rmse)
            r2_scores.append(r2)

        evaluation_scores.append({
            'Random Seed': seed,
            'Batch Size': batch_size,
            'Mean Squared Error (MSE)': mse_scores,
            'Mean Absolute Error (MAE)': mae_scores,
            'Root Mean Squared Error (RMSE)': rmse_scores,
            'R-squared (R2)': r2_scores
        })

        for i, column in enumerate(target_columns):
            print(f"Target Column {column}:")
            print("Mean Squared Error (MSE):", mse_scores[i])
            print("Mean Absolute Error (MAE):", mae_scores[i])
            print("Root Mean Squared Error (RMSE):", rmse_scores[i])
            print("R-squared (R2):", r2_scores[i])
            print()

# Final Evaluation
print("Final Evaluation:")
for evaluation in evaluation_scores:
    print(f"Random Seed: {evaluation['Random Seed']}, Batch Size: {evaluation['Batch Size']}")
    for i, column in enumerate(target_columns):
        print(f"Target Column {column}:")
        print("Mean Squared Error (MSE):", evaluation['Mean Squared Error (MSE)'][i])
        print("Mean Absolute Error (MAE):", evaluation['Mean Absolute Error (MAE)'][i])
        print("Root Mean Squared Error (RMSE):", evaluation['Root Mean Squared Error (RMSE)'][i])
        print("R-squared (R2):", evaluation['R-squared (R2)'][i])
        print()


Iterative Modeling: Random Seed = 42, Batch Size = 1000
Target Column 1:
Mean Squared Error (MSE): 1.067400552467252
Mean Absolute Error (MAE): 0.7314912436053136
Root Mean Squared Error (RMSE): 1.033150788833485
R-squared (R2): 0.6614252153899902

Target Column 45:
Mean Squared Error (MSE): 0.5841620941991067
Mean Absolute Error (MAE): 0.5472567898828538
Root Mean Squared Error (RMSE): 0.7643049746005234
R-squared (R2): 0.7763015767962262

Target Column 56:
Mean Squared Error (MSE): 0.6360537328165623
Mean Absolute Error (MAE): 0.6097295726334686
Root Mean Squared Error (RMSE): 0.7975297692353323
R-squared (R2): 0.7380089112615444

Target Column 57:
Mean Squared Error (MSE): 0.8282468154651064
Mean Absolute Error (MAE): 0.657670281759278
Root Mean Squared Error (RMSE): 0.9100806642628478
R-squared (R2): 0.6710042389547466

Final Evaluation:
Random Seed: 42, Batch Size: 1000
Target Column 1:
Mean Squared Error (MSE): 1.067400552467252
Mean Absolute Error (MAE): 0.7314912436053136
Root 

In [111]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

random_seeds = [2]
batch_sizes = [2]

sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

os.chdir(r'D:\Yaw-00')
all_files = glob.glob('*int')
dataframes = []

evaluation_scores = []

for seed in random_seeds:
    for batch_size in batch_sizes:
        print(f"Iterative Modeling: Random Seed = {seed}, Batch Size = {batch_size}")

        for file in all_files:
            if file.endswith('.int'):
                df = read_int(file, sensors)  # Modified to store dataframe in df variable
                dataframes.append(df)  # Append the dataframe to the list
        
        df = pd.concat(dataframes)
        df = df.transpose()

        target_columns = [1, 45, 56, 57]

        models = []
        mse_scores = []
        mae_scores = []
        rmse_scores = []
        r2_scores = []

        for i in range(len(target_columns)):
            train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)

            model = RandomForestRegressor(random_state=seed)
            X_train = train_df.drop(columns=target_columns)
            y_train = train_df[target_columns[i]]
            model.fit(X_train, y_train)

            X_test = test_df.drop(columns=target_columns)
            y_test = test_df[target_columns[i]]
            y_pred = model.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            models.append(model)
            mse_scores.append(mse)
            mae_scores.append(mae)
            rmse_scores.append(rmse)
            r2_scores.append(r2)

        evaluation_scores.append({
            'Random Seed': seed,
            'Batch Size': batch_size,
            'Mean Squared Error (MSE)': np.mean(mse_scores),
            'Mean Absolute Error (MAE)': np.mean(mae_scores),
            'Root Mean Squared Error (RMSE)': np.mean(rmse_scores),
            'R-squared (R2)': np.mean(r2_scores)
        })

        for i, column in enumerate(target_columns):
            print(f"Target Column {column}:")
            print("Mean Squared Error (MSE):", mse_scores[i])
            print("Mean Absolute Error (MAE):", mae_scores[i])
            print("Root Mean Squared Error (RMSE):", rmse_scores[i])
            print("R-squared (R2):", r2_scores[i])
            print()

print("Final Evaluation:")
for evaluation in evaluation_scores:
    print(f"Random Seed: {evaluation['Random Seed']}, Batch Size: {evaluation['Batch Size']}")
    print("Mean Squared Error (MSE):", evaluation['Mean Squared Error (MSE)'])
    print("Mean Absolute Error (MAE):", evaluation['Mean Absolute Error (MAE)'])
    print("Root Mean Squared Error (RMSE):", evaluation['Root Mean Squared Error (RMSE)'])
    print("R-squared (R2):", evaluation['R-squared (R2)'])
    print()


Iterative Modeling: Random Seed = 2, Batch Size = 2


KeyboardInterrupt: 

In [112]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm

random_seeds = [2]
batch_sizes = [2]

sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

os.chdir(r'D:\Yaw-00')
all_files = glob.glob('*int')
dataframes = []

evaluation_scores = []

for seed in random_seeds:
    for batch_size in batch_sizes:
        print(f"Iterative Modeling: Random Seed = {seed}, Batch Size = {batch_size}")

        pbar = tqdm(total=len(all_files), desc='Processing Files')  # Create progress bar

        for file in all_files:
            if file.endswith('.int'):
                df = read_int(file, sensors)
                dataframes.append(df)
                pbar.update(1)  # Increment progress bar

        pbar.close()  # Close progress bar

        df = pd.concat(dataframes)
        df = df.transpose()

        target_columns = [1, 45, 56, 57]

        models = []
        mse_scores = []
        mae_scores = []
        rmse_scores = []
        r2_scores = []

        for i in range(len(target_columns)):
            train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)

            model = RandomForestRegressor(random_state=seed)
            X_train = train_df.drop(columns=target_columns)
            y_train = train_df[target_columns[i]]
            model.fit(X_train, y_train)

            X_test = test_df.drop(columns=target_columns)
            y_test = test_df[target_columns[i]]
            y_pred = model.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            models.append(model)
            mse_scores.append(mse)
            mae_scores.append(mae)
            rmse_scores.append(rmse)
            r2_scores.append(r2)

        evaluation_scores.append({
            'Random Seed': seed,
            'Batch Size': batch_size,
            'Mean Squared Error (MSE)': np.mean(mse_scores),
            'Mean Absolute Error (MAE)': np.mean(mae_scores),
            'Root Mean Squared Error (RMSE)': np.mean(rmse_scores),
            'R-squared (R2)': np.mean(r2_scores)
        })

        for i, column in enumerate(target_columns):
            print(f"Target Column {column}:")
            print("Mean Squared Error (MSE):", mse_scores[i])
            print("Mean Absolute Error (MAE):", mae_scores[i])
            print("Root Mean Squared Error (RMSE):", rmse_scores[i])
            print("R-squared (R2):", r2_scores[i])
            print()

print("Final Evaluation:")
for evaluation in evaluation_scores:
    print(f"Random Seed: {evaluation['Random Seed']}, Batch Size: {evaluation['Batch Size']}")
    print("Mean Squared Error (MSE):", evaluation['Mean Squared Error (MSE)'])
    print("Mean Absolute Error (MAE):", evaluation['Mean Absolute Error (MAE)'])
    print("Root Mean Squared Error (RMSE):", evaluation['Root Mean Squared Error (RMSE)'])
    print("R-squared (R2):", evaluation['R-squared (R2)'])
    print()


Iterative Modeling: Random Seed = 2, Batch Size = 2


Processing Files:  42%|█████████████████████████▋                                   | 796/1892 [22:06<31:08,  1.70s/it]

KeyboardInterrupt: 

In [118]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm_notebook as tqdm

random_seeds = [42]
batch_sizes = [200]
selected_percentage = 0.02  # Select 2% random rows from each file

sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

os.chdir(r'D:\Yaw-00')
all_files = glob.glob('*int')
dataframes = []

evaluation_scores = []



for seed in random_seeds:
    for batch_size in batch_sizes:
        print(f"Iterative Modeling: Random Seed = {seed}, Batch Size = {batch_size}")

        pbar_files = tqdm(total=len(all_files), desc='Processing Files')  # Create progress bar for files

        for file in all_files:
            if file.endswith('.int'):
                df = read_int(file, sensors)
                if df is not None:  # Check if the DataFrame is not None
                    num_rows = int(df.shape[0] * selected_percentage)  # Calculate number of rows to select
                    df = df.sample(n=num_rows, random_state=seed)  # Randomly select the specified percentage of rows
                    dataframes.append(df)
                pbar_files.update(1)  # Increment progress bar for files

        pbar_files.close()  # Close progress bar for files

        if len(dataframes) == 0:
            print("No dataframes available. Skipping modeling.")
            continue

        df = pd.concat(dataframes)
        df = df.transpose()

        target_columns = [1, 45, 56, 57]

        models = []
        mse_scores = []
        mae_scores = []
        rmse_scores = []
        r2_scores = []

        pbar_models = tqdm(total=len(target_columns), desc='Modeling Progress')  # Create progress bar for models

        for i in range(len(target_columns)):
            train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)

            model = RandomForestRegressor(random_state=seed)
            X_train = train_df.drop(columns=target_columns)
            y_train = train_df[target_columns[i]]
            model.fit(X_train, y_train)

            X_test = test_df.drop(columns=target_columns)
            y_test = test_df[target_columns[i]]
            y_pred = model.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            models.append(model)
            mse_scores.append(mse)
            mae_scores.append(mae)
            rmse_scores.append(rmse)
            r2_scores.append(r2)

            pbar_models.update(1)  # Increment progress bar for models

        pbar_models.close()  # Close progress bar for models

        evaluation_scores.append((models, mse_scores, mae_scores, rmse_scores, r2_scores))

print("Modeling complete. Evaluation scores:")
for eval_scores in evaluation_scores:
    # Extract the evaluation scores for each iteration and print them
    models, mse_scores, mae_scores, rmse_scores, r2_scores = eval_scores
    print("MSE:", mse_scores)
    print("MAE:", mae_scores)
    print("RMSE:", rmse_scores)
    print("R2:", r2_scores)


Iterative Modeling: Random Seed = 42, Batch Size = 200


C:\Users\musab\AppData\Local\Temp\ipykernel_628\2402220438.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar_files = tqdm(total=len(all_files), desc='Processing Files')  # Create progress bar for files


Processing Files:   0%|          | 0/1892 [00:00<?, ?it/s]

Processing Files:   1%|▌                                                           | 19/1892 [02:07<3:29:09,  6.70s/it]


KeyboardInterrupt: 

In [125]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

random_seed = 42
selected_percentage = 0.002

sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

os.chdir(r'D:\Yaw-00')
all_files = glob.glob('*.int')
dataframes = []

evaluation_scores = []

# Manually set the number of iterations
num_iterations = 5

for iteration in range(num_iterations):
    print(f"Iteration {iteration + 1}:")

    for file in all_files:
        if file.endswith('.int'):
            df = read_int(file, sensors)
            if df is not None:  # Check if the DataFrame is not None
                num_rows = int(df.shape[0] * selected_percentage)  # Calculate number of rows to select
                df = df.sample(n=num_rows, random_state=random_seed)  # Randomly select the specified percentage of rows
                dataframes.append(df)

    if len(dataframes) == 0:
        print("No dataframes available. Skipping modeling.")
    else:
        df = pd.concat(dataframes)
        df = df.transpose()

        target_columns = [1, 45, 56, 57]

        models = []
        mse_scores = []
        mae_scores = []
        rmse_scores = []
        r2_scores = []

        for target_column in target_columns:
            train_df, test_df = train_test_split(df, test_size=0.2, random_state=random_seed)

            model = RandomForestRegressor(random_state=random_seed)
            X_train = train_df.drop(columns=target_column)
            y_train = train_df[target_column]
            model.fit(X_train, y_train)

            X_test = test_df.drop(columns=target_column)
            y_test = test_df[target_column]
            y_pred = model.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)

            models.append(model)
            mse_scores.append(mse)
            mae_scores.append(mae)
            rmse_scores.append(rmse)
            r2_scores.append(r2)

        evaluation_scores.append((models, mse_scores, mae_scores, rmse_scores, r2_scores))

    # Clear the dataframes list for the next iteration
    dataframes = []

print("Modeling complete. Evaluation scores:")
for iteration, eval_scores in enumerate(evaluation_scores):
    # Extract the evaluation scores for each iteration and print them
    models, mse_scores, mae_scores, rmse_scores, r2_scores = eval_scores
    print(f"Iteration {iteration + 1}:")
    print("MSE:", mse_scores)
    print("MAE:", mae_scores)
    print("RMSE:", rmse_scores)
    print("R2:", r2_scores)


Iteration 1:


KeyboardInterrupt: 

In [129]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm_notebook as tqdm

random_seed = 42
selected_percentage = 0.002

sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 45, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

os.chdir(r'D:\Yaw-00')
all_files = glob.glob('*.int')
dataframes = []

evaluation_scores = []



for file in all_files:
    if file.endswith('.int'):
        df = read_int(file, sensors)
        if df is not None:  # Check if the DataFrame is not None
            num_rows = int(df.shape[0] * selected_percentage)  # Calculate number of rows to select
            df = df.sample(n=num_rows, random_state=random_seed)  # Randomly select the specified percentage of rows
            dataframes.append(df)

if len(dataframes) == 0:
    print("No dataframes available. Skipping modeling.")
else:
    df = pd.concat(dataframes)
    df = df.transpose()

    target_columns = [1, 45, 56, 57]

    models = []
    mse_scores = []
    mae_scores = []
    rmse_scores = []
    r2_scores = []

    for target_column in target_columns:
        train_df, test_df = train_test_split(df, test_size=0.2, random_state=random_seed)

        model = RandomForestRegressor(random_state=random_seed)
        X_train = train_df.drop(columns=target_column)
        y_train = train_df[target_column]
        model.fit(X_train, y_train)

        X_test = test_df.drop(columns=target_column)
        y_test = test_df[target_column]
        y_pred = model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        models.append(model)
        mse_scores.append(mse)
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        r2_scores.append(r2)

evaluation_scores.append((models, mse_scores, mae_scores, rmse_scores, r2_scores))

print("Modeling complete. Evaluation scores:")
for eval_scores in evaluation_scores:
    # Extract the evaluation scores for each iteration and print them
    models, mse_scores, mae_scores, rmse_scores, r2_scores = eval_scores
    print("MSE:", mse_scores)
    print("MAE:", mae_scores)
    print("RMSE:", rmse_scores)
    print("R2:", r2_scores)


No dataframes available. Skipping modeling.
Modeling complete. Evaluation scores:
MSE: [1.067400552467252, 0.5841620941991067, 0.6360537328165623, 0.8282468154651064]
MAE: [0.7314912436053136, 0.5472567898828538, 0.6097295726334686, 0.657670281759278]
RMSE: [1.033150788833485, 0.7643049746005234, 0.7975297692353323, 0.9100806642628478]
R2: [0.6614252153899902, 0.7763015767962262, 0.7380089112615444, 0.6710042389547466]


In [116]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set the random seed for reproducibility
random_seed = 42

# Set the sensor numbers you want to process
sensors = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

# Set the batch size (number of records to process at a time)
batch_size = 1000

# Read the .int files and create the dataframe
os.chdir(r'C:\Users\musab\raw\Yaw00')
all_files = glob.glob('*int')
dataframes = []


# Process the files in smaller batches
for file in all_files:
    if file.endswith('.int'):
        df = read_int(file, sensors)
        dataframes.append(df)

# Concatenate all dataframes into a single dataframe
df = pd.concat(dataframes)

# Transpose the dataframe
df = df.transpose()

# Set the target columns
target_columns = [1, 45, 56, 57]

# Set the sample size for random sampling
sample_size = 1000

# Set the split ratios for train, validation, and test sets
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Perform random sampling
df_sample = df.sample(n=sample_size, random_state=random_seed)

# Split the sampled dataset into train, validation, and test sets
train_df, val_test_df = train_test_split(df_sample, test_size=val_ratio+test_ratio, random_state=random_seed)
val_df, test_df = train_test_split(val_test_df, test_size=test_ratio/(val_ratio+test_ratio), random_state=random_seed)

# Perform iterative modeling
models = []
mse_scores = []
mae_scores = []
rmse_scores = []
r2_scores = []

for i, column in enumerate(target_columns):
    # Split the train and validation data further for iterative modeling
    train_data, val_data = train_test_split(train_df, test_size=val_ratio, random_state=random_seed)

    # Train the model
    model = RandomForestRegressor(random_state=random_seed)
    X_train = train_data.drop(columns=[column])
    y_train = train_data[column]
    model.fit(X_train, y_train)

    # Evaluate on the validation set
    X_val = val_data.drop(columns=[column])
    y_val = val_data[column]
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, y_pred)

    models.append(model)
    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

# Select the best model based on the validation set performance
best_model_index = np.argmin(mse_scores)
best_model = models[best_model_index]

# Evaluate the best model on the test set
X_test = test_df.drop(columns=target_columns)
y_test = test_df[target_columns]
y_pred = best_model.predict(X_test)

mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred)

# Print evaluation scores for each target column
for i, column in enumerate(target_columns):
    print(f"Target Column {column}:")
    print("Iterative Modeling - Validation Set:")
    print("Mean Squared Error (MSE):", mse_scores[i])
    print("Mean Absolute Error (MAE):", mae_scores[i])
    print("Root Mean Squared Error (RMSE):", rmse_scores[i])
    print("R-squared (R2):", r2_scores[i])
    print()

print("Final Evaluation - Test Set:")
print("Mean Squared Error (MSE):", mse_test)
print("Mean Absolute Error (MAE):", mae_test)
print("Root Mean Squared Error (RMSE):", rmse_test)
print("R-squared (R2):", r2_test)


ValueError: All objects passed were None